# Part 2: Fitting RL Models - Understanding Human Decision-Making

## Introduction: Why Computational Modeling Matters

In this tutorial, we will learn how to implement and evaluate computational models of human decision-making. This is a crucial skill for understanding how people learn, make choices, and adapt their behavior based on experience.

### The Scientific Question

Imagine you're a researcher studying how people learn to make optimal choices. You observe participants playing a simple game where they repeatedly choose between two slot machines with different (unknown) reward probabilities. Some participants learn quickly, others struggle. Some are cautious, others take risks. 

**The key question**: What cognitive strategies do people use to maximize their rewards? Are they using simple heuristics, sophisticated learning algorithms, or something in between?

### The Wilson & Collins Framework

This tutorial is based on the influential paper:
**Wilson & Collins (2019) eLife: Ten simple rules for the computational modeling of behavioral data**

The authors provide a systematic approach to computational modeling that involves:
1. **Model Construction**: Building mathematical models of cognitive processes
2. **Parameter Estimation**: Finding the best-fitting parameters for each individual
3. **Model Comparison**: Determining which model best explains the data
4. **Validation**: Ensuring our methods are reliable and meaningful

### The 2-Armed Bandit Task

We'll use a classic experimental paradigm: the 2-armed bandit task. In this task:
- Participants perform $T$ choices between two options (slot machines)
- The machines have asymmetric reward probabilities: $\mu_{1} = 0.2$ and $\mu_{2} = 0.8$
- These probabilities are **unknown** to the participant
- The goal is to maximize total reward over time

### What You'll Learn

By the end of this tutorial, you will:
1. **Understand** three different models of human decision-making
2. **Implement** behavioral analyses to compare model predictions
3. **Validate** your modeling approach using parameter recovery
4. **Evaluate** model identifiability using model recovery
5. **Apply** these techniques to understand real human behavior

### The Bigger Picture

This tutorial demonstrates fundamental principles of computational cognitive science. These methods are used to:
- Understand psychiatric conditions
- Evaluate educational interventions
- Design better human-computer interfaces
- Study neural mechanisms of learning and decision-making

---

## Setup: Importing Libraries and Data

Let's start by importing the necessary modules and setting up our computational environment. We'll be working with numerical data, creating visualizations, and implementing optimization algorithms.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

# Download the custom library - do this manually if not running on google colab
MODULE_NAME = "models.py"
if not os.path.isfile("models.py"):
    MODULE_URL = f'https://raw.githubusercontent.com/bambschool/BAMB2025/main/Day_2_reinforcement_learning/part2_fitting_rl_models/{MODULE_NAME}'
    !wget -O {MODULE_NAME} "{MODULE_URL}"

ModuleNotFoundError: No module named 'scipy'

## 1. Experimental Setup

Before we dive into modeling, let's establish the experimental parameters that define our 2-armed bandit task. These parameters will be used throughout the tutorial to generate synthetic data and test our models.

In [3]:
# set numpy seed to 0 for reproducibility
np.random.seed(0)

In [4]:
# Experiment parameters
num_trials = 100
reward_probabilities = np.array([0.2, 0.8])
num_repetitions = 110

## 2. Three Models of Human Decision-Making

Now we'll explore three different computational models that capture different theories about how humans make decisions in uncertain environments. Each model represents a different cognitive strategy that people might use when faced with the 2-armed bandit task.

### Understanding the Models

**Why three models?** Different people might use different strategies, and the same person might use different strategies in different contexts. By comparing multiple models, we can gain insights into the diversity of human decision-making processes.

**Key insight**: These models make different predictions about behavior, which we can test against real data to understand which cognitive processes are most likely at play.

### The Three Candidate Models

We will consider three different computational models that represent distinct hypotheses about human decision-making:

#### Model 1: **Noisy Win-Stay-Lose-Shift (WSLS)**
- **Core idea**: Simple heuristic strategy
- **Mechanism**: If the last choice was rewarded → repeat it; if not → switch to the other option
- **Randomness**: Occasionally explores randomly (parameter: $\epsilon$)
- **Psychological insight**: Represents basic associative learning without sophisticated value computation
- **Real-world example**: Like a person who sticks with a restaurant if they had a good meal, switches if they had a bad meal

#### Model 2: **Rescorla-Wagner Learning**
- **Core idea**: Sophisticated value learning based on prediction errors
- **Mechanism**: Maintains estimates of each option's value, updates based on reward prediction error
- **Parameters**: Learning rate $\alpha_{RW}$ (how fast you learn) and softmax temperature $\beta_{RW}$ (how deterministic your choices are)
- **Psychological insight**: Represents gradual learning and memory integration
- **Real-world example**: Like carefully tracking your experiences with different restaurants and slowly updating your preferences

#### Model 3: **Choice Kernel**
- **Core idea**: Tendency to repeat recent actions regardless of outcomes
- **Mechanism**: Builds up "momentum" for recently chosen actions
- **Parameters**: Kernel learning rate $\alpha_{CK}$ and temperature $\beta_{CK}$
- **Psychological insight**: Captures perseveration and habit formation
- **Real-world example**: Like having a favorite restaurant you keep going to out of habit, even when the food quality varies

### Key Differences

These models make different predictions about:
- **Sensitivity to rewards**: WSLS and Rescorla-Wagner are outcome-sensitive, Choice Kernel is not
- **Learning speed**: Different models learn at different rates
- **Exploration vs. exploitation**: Models balance these differently
- **Individual differences**: People might use different strategies

### 2.1. Initialize the Models

The models are already implemented in `models.py`. Let's import and initialize them. Take a moment to examine the code structure in `models.py` - notice how each model inherits from a base class and implements the same interface (simulate, fit, etc.).

In [5]:
from models import WinStayLoseSwitch, RescorlaWagner, ChoiceKernel

# Initialize models
wsls_model = WinStayLoseSwitch()
rw_model = RescorlaWagner()
ck_model = ChoiceKernel()

ModuleNotFoundError: No module named 'scipy'

### 2.1.1. (Optional) Deep Dive: Implement Rescorla-Wagner Yourself

**Learning objective**: Understand the mathematical details of the Rescorla-Wagner model by implementing it from scratch.

While all models are provided in `models.py`, implementing the Rescorla-Wagner model yourself will deepen your understanding of how computational models work.

**The Rescorla-Wagner learning rule**:
- Start with initial value estimates: $Q_0 = [0.5, 0.5]$ (neutral expectations)
- For each trial $t$:
  1. **Choice**: Select action based on softmax: $P(a) = \frac{e^{\beta Q_a}}{\sum_{a'} e^{\beta Q_{a'}}}$
  2. **Outcome**: Observe reward $r_t$ 
  3. **Learning**: Update value with prediction error: $Q_a \leftarrow Q_a + \alpha(r_t - Q_a)$

**Key insights**:
- $\alpha$ controls learning speed (0 = no learning, 1 = only remember last outcome)
- $\beta$ controls choice determinism (0 = random, ∞ = always choose best)
- The model learns from prediction errors, not just outcomes

**Your task**: Fill in the methods below and compare with the implementation in `models.py`.

In [ ]:
from models import RLModel

class RescorlaWagner(RLModel):
    def simulate(self, T, mu, alpha, beta):
        # initialize Q values, actions, rewards
        Q = np.array([0.5, 0.5])
        a, r = [], []

        # for each timestep:
        #   compute choice probability, action, reward
        #   append action and reward to respective lists
        #   update the Q value
        # return the list of actions and rewards

        return np.array(a), np.array(r)

    def likelihood(self, pars, a, r):
        alpha, beta = pars
        Q = np.array([0.5, 0.5])
        choice_p = []

        # for each timestep:
        #   compute action probability
        #   append it to the choice_p list
        #   update the Q value
        # return the negative log likelihood

    def initial_parameters(self):
        return [np.random.random(), np.random.exponential()]

    def parameter_bounds(self):
        return [(0, 1), (0, np.inf)]

### 2.2. Simulate the Models

**Learning objective**: Generate synthetic behavioral data from our three models to understand their predictions.

**Why simulate?** Before we can fit models to real data, we need to understand what behavior each model predicts. Simulation allows us to:
- Test our implementation
- Understand parameter effects
- Generate ground truth data for validation
- Explore the model's behavioral repertoire

**Your task**: Implement the `simulate_model()` function that will generate synthetic choice sequences for each model. The function should:
- Take a model instance and parameters
- Run multiple simulations (repetitions)
- Return lists of actions and rewards for analysis

**Parameter choices**: We'll use these parameters for our simulations:
- **Win Stay Lose Shift**: $\epsilon=0.1$ (10% random exploration)
- **Rescorla Wagner**: $\alpha_{RW}=0.1$ (slow learning), $\beta_{RW}=3$ (moderately deterministic)
- **Choice Kernel**: $\alpha_{CK}=0.1$ (slow habit formation), $\beta_{CK}=3$ (moderately deterministic)

**Think about**: How do you expect these different models to behave? Which should perform best? Which should be most/least sensitive to rewards?

In [1]:
# define a function to simulate the models and collect data
def simulate_model(model, num_reps, num_trials, reward_probs, **params):
    actions, rewards = [], []
    # for each repitition
    #   simiulate models
    #   append actions and rewards to the respective lists
    # return actions, rewards

# Simulate models
a_wsls, r_wsls = simulate_model(wsls_model, num_repetitions, num_trials, reward_probabilities, epsilon=0.1)
a_rw, r_rw = simulate_model(rw_model, num_repetitions, num_trials, reward_probabilities, alpha=0.1, beta=5)
a_ck, r_ck = simulate_model(ck_model, num_repetitions, num_trials, reward_probabilities, alpha=0.1, beta=3)

NameError: name 'wsls_model' is not defined

For each model, you should also play with the parameters used to generate the simulations and observe the effect on Win-Stay-Lose-Shift analysis (see below). 

## 3. Behavioral Analysis: Understanding Model Predictions

**Learning objective**: Learn to analyze and visualize model behavior to understand what makes different models distinct.

**Why behavioral analysis?** Before fitting models to real data, we need to understand what patterns each model predicts. This helps us:
- Identify which models are actually different from each other
- Understand what behavioral signatures to look for in real data
- Design better experiments that can distinguish between models
- Build intuition about model behavior

**The key insight**: If two models predict identical behavior, there's no point in comparing them. Good computational modeling requires models that make different, testable predictions.

### 3.1. Win-Stay-Lose-Shift Analysis

**What is WSLS analysis?** Win-Stay-Lose-Shift analysis is a fundamental tool in behavioral neuroscience that measures how much a decision-maker's choices depend on the outcomes of their previous actions.

**Why is this useful?** WSLS analysis reveals:
- **Outcome sensitivity**: How much do previous rewards influence future choices?
- **Model differences**: Different models show distinct WSLS patterns
- **Individual differences**: People vary in their WSLS behavior
- **Clinical applications**: WSLS patterns change in psychiatric conditions

**The measurement**: We calculate:
- $p(stay|win)$: Probability of repeating an action after it was rewarded
- $p(stay|lose)$: Probability of repeating an action after it was unrewarded

**Expected patterns**:
- **WSLS model**: Should show stark differences (high p(stay|win), low p(stay|lose))
- **Rescorla-Wagner**: Should show moderate outcome sensitivity
- **Choice Kernel**: Should show similar p(stay) regardless of previous reward

This analysis reproduces **Figure 1** from Wilson & Collins (2019), demonstrating how different models produce distinct behavioral signatures.

### 3.1. Compare qualitative patterns from our three different models

### 3.1.1. Implementing WSLS Analysis

**Your task**: Implement the `compute_stay_probabilities()` function that calculates WSLS measures from behavioral data.

**Algorithm**:
1. For each trial $t > 1$:
   - Determine if the participant "stayed" (chose the same action as trial $t-1$)
   - Check if the previous trial was rewarded or unrewarded
2. Calculate:
   - $p(stay|win) = \frac{\text{number of stays after wins}}{\text{total number of wins}}$
   - $p(stay|lose) = \frac{\text{number of stays after losses}}{\text{total number of losses}}$

**Implementation hints**:
- Use `np.hstack()` to handle the first trial (which has no previous trial)
- Be careful with indexing - you're comparing trial $t$ with trial $t-1$
- Use `np.nanmean()` to handle missing values appropriately

**What to expect**: After implementing this function, you should be able to reproduce the characteristic WSLS patterns that distinguish the three models.

In [9]:
def compute_stay_probabilities(actions, rewards):
    """Compute probability of stay given win/lose."""
    # return lose_stay, win_stay

In [10]:
# Calculate WSLS for each model
model_actions = [a_wsls, a_rw, a_ck]
model_rewards = [r_wsls, r_rw, r_ck]
model_names = ['WSLS', 'Rescorla-Wagner', 'Choice kernel']

wsls_probs = []
for actions, rewards in zip(model_actions, model_rewards):
    wsls_probs.append(np.mean([compute_stay_probabilities(actions[n], rewards[n]) for n in range(num_repetitions)], axis=0))

# Loop over your models
A = [a_wsls, a_rw, a_ck]
R = [r_wsls, r_rw, r_ck]

wsls_probs = []
for actions, rewards in zip(model_actions, model_rewards):
    wsls_probs.append(np.mean([compute_stay_probabilities(actions[n], rewards[n]) for n in range(num_repetitions)], axis=0))

Now plot WSLS behavior as a function of previous reward (1 for rewarded, 0 for unrewarded).

- Does each model modulate its probability of staying as a function of previous reward? 

In [5]:
# Plot WSLS as a function of previous reward
plt.figure(figsize=(4, 4), dpi=100)
for i, prob in enumerate(wsls_probs):
    plt.plot([0, 1], prob, 'o-', label=model_names[i])
plt.xlabel('Previous reward')
plt.ylabel('Probability of staying')
plt.xticks([0, 1])
plt.legend(frameon=False)
plt.ylim(0, 1)

You should see that the choice kernel model leads to a reward-independent $p(stay)$, because choice probabilities are calculated independently of the previous reward. All other models show outcome-modulated behavior, with the starkest differences for the WSLS simulation.

*Take home message*: More broadly, these patterns of behavior can then be contrasted again actual behavioral data to inform about subjects' behavior. It is important to simulate your candidate models and plot their behavior before comparing them to actual data. 

### 3.2. Parameter Effects Analysis: Understanding the Rescorla-Wagner Model

**Learning objective**: Understand how model parameters affect behavior and performance.

**Why study parameter effects?** Different parameter values lead to different behavioral patterns. Understanding these relationships helps us:
- Interpret fitted parameters meaningfully
- Design better experiments
- Understand individual differences
- Predict the effects of interventions

**The analysis**: We'll systematically vary the Rescorla-Wagner parameters ($\alpha$ and $\beta$) and measure how they affect performance in early vs. late trials.

**Key questions**:
- How does learning rate ($\alpha$) affect early learning vs. asymptotic performance?
- How does temperature ($\beta$) interact with learning rate?
- Are there optimal parameter combinations?
- What do these patterns tell us about the explore-exploit tradeoff?

For Rescorla Wagner, we are now interested in how learning rate and softmax inverse temperate affect the probability of choosing the arm with the highest reward. 

We will repeatedly perform a grid search over different parameter values (~ 1000 simulations with 100 trials per grid point) and store the mean $p(correct)$ across trials for each grid point and repetition.

In [6]:
# Parameter grid search for Rescorla-Wagner model
alphas = np.linspace(0.02, 1, 4)
betas = np.array([1, 2, 5, 10, 20])

In [7]:
# Let's first use only 10 simulations for each parameter combination. When your code works, increase to 1000.
num_reps = 1_000

# Initialize arrays to collect data
correct = np.zeros((len(alphas), len(betas), num_reps))
correct_early = np.zeros((len(alphas), len(betas), num_reps))
correct_late = np.zeros((len(alphas), len(betas), num_reps))

# Evaluation loop: grid-search over alpha and beta parameters for a large number of simulations
# on which you will then average.
    # simulate rescorla wagner model and collect sequence of actions and rewards
    # collect the performance
    # store performance for early (first 10 trials) and late (last 10 trials of a block) trials

Now plot $p(correct)$ as a function of $\alpha$ and $\beta$. Create the figure with two subplots: one for early, one for late trials.

As in Wilson & Collins box 2 figure 1, plot different levels of $\alpha$ on the x-axis and use different curves for $\beta$ levels.

In [8]:
# Your figure with two subplots here

How does performance change as a function of alpha and beta parameter values, for early and late trials?

The left graph shows that the learning rate is positively correlated with increases in early performance only for low $\beta$ values, or very low $\alpha$ values. For high $\beta$ values, there is a U-shaped relationship between learning rate and early speed of learning. The right graph shows that with high $\beta$ values, high learning rates negatively influence asymptotic behavior. Thus, both parameters interact to influence both the speed of learning and asymptotic performance. 

*Conclusion*: This kind of analysis will allow you to see qualitative differences between models, so that making their predictions in the experimental setup different. If the behavior of different models is not qualitatively different, this is a sign that you should try to design a better experiment. While not always possible, distinguishing between models on the basis of qualitative patterns in the data is preferable to quantitative model comparison. 

## 4. Parameter Recovery: Validating Our Methods

**Learning objective**: Learn to validate computational modeling methods using parameter recovery.

**What is parameter recovery?** Parameter recovery tests whether our fitting procedure can accurately recover the true parameters used to generate synthetic data. This is crucial because:
- It validates that our fitting method works correctly
- It reveals parameter ranges where fitting is reliable
- It identifies potential biases in our estimation procedure
- It's required before applying methods to real data

**The logic**:
1. **Generate**: Use known parameters to simulate behavioral data
2. **Fit**: Apply our fitting procedure to recover the parameters
3. **Compare**: Check how well recovered parameters match the true parameters

**Why this matters**: If we can't recover known parameters from synthetic data, we can't trust our method when applied to real data where the true parameters are unknown.

**Wilson & Collins Rule #8**: "Parameter recovery is essential for validating your computational modeling approach."

### 4.1. Simulation and fitting

### 4.1. Parameter Recovery Procedure

**The systematic approach**:
1. **Sample parameters**: Draw random parameter values from realistic distributions
2. **Simulate behavior**: Generate synthetic data using these parameters
3. **Fit model**: Apply maximum likelihood estimation to recover parameters
4. **Evaluate**: Compare recovered vs. true parameters

**What to look for**:
- **Correlation**: Strong correlation between true and recovered parameters
- **Bias**: Systematic over- or under-estimation
- **Range effects**: Whether recovery quality depends on parameter values
- **Noise**: How much variability there is in recovery

**Implementation strategy**: We'll focus on the Rescorla-Wagner model, using realistic parameter ranges and sufficient trial numbers to ensure reliable fitting.

We will do this step by step. We will first simulate actions of the model for a given learning rate, $\alpha$, and softmax parameter, $\beta$.			

After simulating the model, we will fit the parameters using a maximum likelihood approach to get estimated values $\hat{\alpha}$ and $\hat{\beta}$. 

In [14]:
# experiment parameters
num_reps = 100
alphas = np.random.rand(num_reps)
betas = 10 * np.random.exponential(size=num_reps)

Fit the simulated data. 

Fit Rescorla Wagner by calling the `fit()` method for that model, `rw_model.fit()`, which takes as inputs actions and rewards. We will not focus on the goodness of fit here, but you should take a moment to look at the specification in `.fit()` and `likelihood()` methods of the model in `models.py`.

In [9]:
# sample random parameters

# loop over repetitions 

    # set different fixed seeds per repetition
    
    # simulate M3

    # fit and store parameters


### 4.1. Parameter recovery plots

We would like to visualise the fitted parameter values as a function of the generating parameter values. We will have one data point for each iteration. Here, you should use the generating values and the fitted values that you have stored in the section above. You should create two subplots, one for each parameter.

In [10]:
# Your figure with two subplots goes here


Do you observe a fairly good agreement between the simulated and fit parameter values?

The plot makes any correlations clear, and also reveals whether the correlation holds in some parameter regimes but not others. It also reveals any existing bias (e.g. a tendency to recover higher or lower values in average).

Here we can see that the fit for $\beta$ is best with a range, $0.1 < \beta < 10$ and that outside this range, the correspondence between simulation and fit is not as good.

Depending on the values of $\beta$ that we obtain when fitting human behavior, this worse correspondence at small and large $\beta$ may or may not be problematic. It may be a good idea to use the range of parameters obtained from fitting the real data to test the quality of recovery within the range that matters. 

Reliable parameter recovery is particularly important for look at inter-individual differences in relation to questionnaire scores or brain data.

## 5. Model Recovery: Testing Model Identifiability

**Learning objective**: Learn to test whether different models can be reliably distinguished from each other.

**What is model recovery?** Model recovery (also called model identifiability) tests whether we can correctly identify which model generated a given dataset. This is essential because:
- It validates that our models are actually different from each other
- It reveals when models are confusable
- It helps design better experiments
- It's required before making claims about which model best fits real data

**The confusion matrix approach**:
1. **Simulate**: Generate data from each model using realistic parameters
2. **Fit all models**: Apply all models to each simulated dataset
3. **Select best**: Choose the model with the best fit (lowest BIC)
4. **Tabulate**: Create a confusion matrix showing recovery rates

**Reading the confusion matrix**:
- **Rows**: True generating model
- **Columns**: Best-fitting model
- **Diagonal**: Correct model identification
- **Off-diagonal**: Model confusability

**Wilson & Collins Rule #9**: "Test model recovery to ensure your models are distinguishable."

To illustrate model recovery, here we will simulate behavior of our three models on the two-armed bandits task. 

As before, the means $\mu$ can be set at 0.2 and 0.8, and the number of trials at $T = 1000$. For each simulation, model parameters can be sampled randomly for each model. 

Each simulated data set will then be fit to each of the given models, to determine which model fit best (according to BIC). This process will be repeated 100 times (number of "repetitions" or "counts") to compute the confusion matrix.

In [11]:
# Model comparison
num_trials = 1_000
num_counts = 10

### 5.1. Building the Confusion Matrix

**Implementation approach**:
1. **For each repetition**:
   - For each of the 3 models, generate synthetic data with random parameters
   - Fit all 3 models to each dataset
   - Record which model fits best (lowest BIC)
2. **Tabulate results**: Count how often each model is correctly identified

**What makes a good confusion matrix?**
- **Strong diagonal**: High values on the diagonal indicate good model recovery
- **Weak off-diagonal**: Low values off the diagonal indicate models are distinguishable
- **Balanced**: All models should be recoverable, not just one

**Parameter considerations**: The confusion matrix depends heavily on the parameter ranges used for simulation. Parameters should match those observed in real human data when possible.

**Troubleshooting**: If you get poor model recovery, consider:
- Increasing the number of trials
- Adjusting parameter ranges
- Using different model comparison criteria
- Checking if models are actually different enough

In [12]:
# models to loop through
models = [wsls_model, rw_model, ck_model]

# initialise your confusion matrix: 3 by 3 (for our three models).
confusion_matrix = np.zeros((3, 3))

# Let's loop over number of repetitions: start with 10, increase to 100 if everything works

    # set different fixed seed for each repetition

    # for each model
    #   simulate to get actions and rewards

    # fit models
    # compute best model and confusion matrix    


In [13]:
# normalize confusion matrix


In [14]:
# plot and print values


### 5.2. Interpreting Model Recovery Results

**Understanding the confusion matrix**:
- **Perfect recovery**: Identity matrix (1s on diagonal, 0s elsewhere)
- **Poor recovery**: Uniform matrix (all values around 0.33)
- **Systematic bias**: Consistent mis-identification patterns

**Critical questions to consider**:
- Are all models recoverable with reasonable accuracy (>70% on diagonal)?
- Are there systematic confusions between specific model pairs?
- How does changing parameters affect model recovery?
- Are the parameter ranges realistic for human behavior?

**Real-world implications**:
- **Good recovery**: You can trust model comparison results on real data
- **Poor recovery**: Models may be too similar or experiment too short
- **Parameter-dependent recovery**: Need to match simulation parameters to real data

**Wilson & Collins insight**: "Models that are identifiable in one parameter regime may be impossible to distinguish in another regime."

**Final considerations**:
- Model recovery is just one validation step - you should also check parameter recovery
- The confusion matrix only tells you about the models you tested - there may be other models that fit better
- Always start with good models that capture competing scientific hypotheses
- Consider both qualitative (behavioral patterns) and quantitative (model comparison) approaches

**Congratulations!** You've now completed a full computational modeling workflow following the Wilson & Collins framework. You're ready to apply these methods to real behavioral data!